In [4]:
import sys
sys.path.append("/home/ubuntu/mlab/days/w3d3/rl_env/lib/python3.8/site-packages")
sys.path.append("/home/ubuntu/mlab/days/w3d3/rl_env/lib/python3.9/site-packages")

In [36]:
import gym
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
from IPython.display import Video
#from video_recorder import VideoRecorder
import video_recorder
import time

# env_name = "CartPole-v1"
# env_name = "Acrobot-v1"
env_name = "MountainCar-v0"
record = True
video_name = "test_0"

env = gym.make(env_name)
state = env.reset()
done = False
total_reward = 0

def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
if record:
    recorder = video_recorder.VideoRecorder(
        env, f"videos/{video_name}.mp4")

states = 0
while not done:
    states += 1
    if record:
        recorder.capture_frame()
    else:  
        show_state(env)
    state, reward, done, _ = env.step(env.action_space.sample()) # Take a random action
    print(state)
    total_reward += reward

print(f"total reward: {total_reward}")
if record:
    recorder.close()


[-0.54211523  0.00114739]
[-0.54082903  0.0012862 ]
[-0.53841367  0.00241536]
[-0.53488723  0.00352644]
[-0.53227614  0.00261109]
[-0.52859998  0.00367616]
[-0.52388631  0.00471367]
[-0.51917049  0.00471582]
[-0.51448788  0.00468261]
[-0.50987359  0.00461429]
[-0.50536221  0.00451138]
[-0.50098753  0.00437467]
[-0.49678231  0.00420522]
[-0.491778    0.00500431]
[-0.48701198  0.00476602]
[-0.48151981  0.00549217]
[-0.4753424   0.00617741]
[-0.46952564  0.00581675]
[-0.46411266  0.00541298]
[-0.46014346  0.0039692 ]
[-0.4556473   0.00449616]
[-0.45265726  0.00299004]
[-0.45019528  0.00246199]
[-0.44827938  0.0019159 ]
[-0.44592358  0.0023558 ]
[-0.44514508  0.00077849]
[-0.44394957  0.00119551]
[-0.44234577  0.00160381]
[-4.42345340e-01  4.26921587e-07]
[-0.4439483  -0.00160296]
[-0.44714296 -0.00319467]
[-0.44990603 -0.00276307]
[-0.45421731 -0.00431127]
[-0.4580452  -0.00382789]
[-0.46136157 -0.00331638]
[-0.46514202 -0.00378045]
[-0.47035865 -0.00521663]
[-0.47497288 -0.00461423]
[-0.

In [30]:
# Display a saved video 
Video("videos/test_0.mp4")
